# 2. Introduction to Machine Learning in Python with Scikit-learn
![LISA logo](https://raw.githubusercontent.com/wshand/Python-Data-Science-Workshop/master/assets/LISA_logo_medium.jpg)

This notebook introduces some basic machine learning methods, and how to implement them using the scikit-learn library.

# Table of contents
* [Introduction](#introduction)
* [Support vector machines](#svm)
* [Normalizing data](#normalization)
* [Principal components analysis](#pca)
* [Scikit-learn pipelines](#pipelines)
* [Cross-validation and grid search](#cross-validation)
* [Additional references](#additional-references)

# Introduction <a id="introduction"></a>
In recent years, Python has become a popular programming language for machine learning for a few reasons:

* Python is a high-level language, and fairly easy to learn
* It's relatively easy to integrate with low-level languages like C and Fortran, which can make well-written Python code run extremely quickly.
* Python already had a lot of great libraries for data visualization and manipulation, like pandas and matplotlib

In this workshop, we're going to use the [Labeled Faces in the Wild (LFW) dataset](http://vis-www.cs.umass.edu/lfw/). We'll use a subset of the data that includes about 1,300 images of seven individuals. Run the code cell below to download the data, and show a few of the people in the dataset.

In [ ]:
from sklearn.datasets import fetch_lfw_people
import itertools
import matplotlib.pyplot as plt
import numpy as np

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.3)

# Get the number of images, as well as the height and width (in pixels) of each image
n_samples, h, w = lfw_people.images.shape

# Separate data into images and labels
X = lfw_people.data
y = lfw_people.target

print("Number of faces:", n_samples)
print("Number of unique people:", len(lfw_people.target_names))
print("Height of every image:", h, "pixels")
print("Width of every image:", w, "pixels")

# Plot some randomly selected faces
n_images_x, n_images_y = (5,3)
fig, axes = plt.subplots(n_images_y, n_images_x, figsize=(12,8))
rand_choices = np.random.randint(0, high=n_samples, size=n_images_x * n_images_y)

for (kk, (ii, jj)) in zip(rand_choices, itertools.product(range(n_images_y), range(n_images_x))):
    face = lfw_people.images[kk]
    name = lfw_people.target_names[y[kk]]
    axes[ii,jj].imshow(face, cmap='gray')
    axes[ii,jj].set_xticks([])
    axes[ii,jj].set_yticks([])
    axes[ii,jj].set_xlabel(name)

plt.show()

# Support vector machines
[SVM wiki article](https://en.wikipedia.org/wiki/Support_vector_machine)

# Normalizing data <a id="normalization"></a>
[Normalization wiki article](https://en.wikipedia.org/wiki/Normalization_(statistics))

# Principal components analysis <a id="pca"></a>
Our images are 37 pixels tall by 28 pixels wide, and hence consist of 1,036 pixels each. This means that each data point we are using to train our support vector machine is 1,036-dimensional, which is fairly large. By reducing this dimension via *dimensionality reduction*, we are able to train our SVM much faster.

One common way of performing dimensionality reduction is via [*principal components analysis*](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA). PCA finds the "most important" parts of the data (more specifically, it finds the directions in which the data vary the most), keeps those parts, and throws the rest away. As a demonstration, try out the code cells below.

In [ ]:
from scipy.stats import multivariate_normal
from sklearn.decomposition import PCA

def square_axes(ax, data, expansion_factor=1.05):
    # Change limits of plot axes to center on the input dataset, and to put the
    # x-axis and y-axis on the same scale
    m        = np.mean(data)
    max_dist = max([np.linalg.norm(u-m) for u in data]) * expansion_factor
    lims     = [m-max_dist, m+max_dist]
    try:    ax.set_xlim(lims); ax.set_ylim(lims)
    except: ax.xlim(lims); ax.ylim(lims)

# Create some random data
rdata = multivariate_normal.rvs(cov=[[4, 1], [1, 1]], size=200)

# Fit PCA to the data
pca = PCA(n_components=2)
pca.fit(rdata)

# TODO: Plot data and principal components
plt.scatter(rdata[:,0], rdata[:,1], marker='x', color='k')
square_axes(plt, rdata)
plt.title("Some random data. Principal components shown as red arrows")
plt.show()

In [ ]:
line              = np.max(np.abs(rdata)) * np.array([-pca.components_[0], pca.components_[0]])
projected_lengths = rdata.dot(pca.components_[0])
projections       = np.array([l * pca.components_[0] for l in projected_lengths])

fig, axes = plt.subplots(3, 1, figsize=(8,16))

# Plot data and line through first principal component in first subplot
axes[0].scatter(rdata[:,0], rdata[:,1], marker='x', color='k')
axes[0].plot(line[:,0], line[:,1], color='red', linewidth=4)
axes[0].set_title("Original data, plus line in the direction of the first principal component")

# Plot data projected onto line in the last subplot
axes[2].scatter(projected_data, [0 for d in projected_data], marker='x', color='k')
axes[2].set_title("Data after transformation by PCA. Data are now one-dimensional instead of two-dimensional")

# Show the effect of the projection on the middle subplot using a few of the points. Points
# are chosen randomly from those with the highest residual norm.
c = sorted(range(rdata.shape[0]), key=lambda ii: np.linalg.norm(rdata[ii]-projections[ii]))
c = np.array(c)[-np.random.choice(20, size=8)]
axes[1].scatter(rdata[:,0], rdata[:,1], marker='x', color='k', alpha=0.2)
axes[1].plot(line[:,0], line[:,1], color='red')
axes[1].scatter(projections[c,0], projections[c,1], color='k')
axes[1].scatter(rdata[c,0], rdata[c,1], marker='x', color='k')
axes[1].set_title("Data are transformed by finding the closest point on the line")
for (orig,proj) in zip(rdata[c], projections[c]):
    axes[1].plot([orig[0], proj[0]], [orig[1], proj[1]], 'r--')

for (ax,dat) in zip(axes, (rdata, rdata[c], rdata)):
    ax.set_xticks([]); ax.set_yticks([])
    square_axes(ax,dat)

plt.show()

To get a more tangible idea of what PCA does, we can look at some of the [eigenfaces](https://en.wikipedia.org/wiki/Eigenface) from the LFW dataset.

In [ ]:
# Train PCA. Show some of the eigenfaces
from sklearn.decomposition import PCA, KernelPCA

n_components=n_images_x * n_images_y
pca = PCA(n_components=n_components).fit(X)

# Show some eigenfaces
fig, axes = plt.subplots(n_images_y, n_images_x, figsize=(12,8))
for (kk, (ii,jj)) in enumerate(itertools.product(range(n_images_y), range(n_images_x))):
    eigenface = pca.components_[kk].reshape((h,w))
    axes[ii,jj].imshow(eigenface, cmap='gray')
    axes[ii,jj].set_xticks([])
    axes[ii,jj].set_yticks([])
    axes[ii,jj].set_xlabel("#" + str(kk + 1))

plt.show()

# Bringing everything together: Pipelines in scikit-learn <a id="pipelines"></a>
So far, in order to add new steps to our machine learning system we've had to define a variable for our data transformer/classifier, fit the machine learning algorithm, and then create new variables to pass on to the next step in the system. Keeping track of all of those variables can quickly get complicated. For that reason, scikit-learn includes the [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to store the entire learning system from start to end.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import time

learner = Pipeline([
    ('scaler', StandardScaler()),
    ('dim-reduce', PCA(n_components=100)),
    ('classify', SVC(C=1.0, gamma='scale'))
])

learner.fit(X, y)

# Cross-validation and grid search <a id="cross-validation"></a>
One of the most important parts of all machine learning algorithms is picking the best possible hyperparameters. The difference between a well-chosen and poorly-chosen hyperparameter can be staggering; *compare accuracy of methods after choosing C correctly*

In order to improve our accuracy, we're going to try to find the best value of the hyperparameter `C` that the support vector machine uses. Recall that `C` *describe what C does*.

The easiest way to choose `C` is to split our data into a *training set* and a *test set*. The training set is the part of the data that our machine learning algorithm actually learns from. Once the learner is trained, we score it on the test set, which gives a better estimate of how good our learner really is. After picking a few values of `C` and training our learner, we pick the value of `C` that got the best score on the test set.

In [ ]:
from sklearn.model_selection import train_test_split

learner = Pipeline([
    ('scaler', StandardScaler()),
    ('dim-reduce', PCA(n_components=250)),
    ('classify', SVC(C=1.0, gamma='scale', kernel='rbf'))
])

# Split into training and tests sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# Fit the learner and score on the training and testing sets
learner.fit(X_train, y_train)
print("Score on training set: %.4f" % learner.score(X_train, y_train))
print("Score on testing set: %.4f" % learner.score(X_test, y_test))

*Cross-validation*

In [ ]:
from sklearn.model_selection import KFold

learner = Pipeline([
    ('scaler', StandardScaler()),
    ('dim-reduce', PCA(n_components=100)),
    ('classify', SVC(C=1.0, gamma='scale'))
])

kf = KFold(n_splits=5)
scores = []
for (train, valid) in kf.split(X_train):
    learner.fit(X_train[train], y_train[train])
    scores.append(learner.score(X_train[valid], y_train[valid]))

print("Average score on validation sets: %.4f" % np.mean(scores))

*Grid search*

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'classify__C': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]}
grid_search = GridSearchCV(learner, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Additional References <a id="additional-references"></a>
* O'Reilly books:
* Since the boom of interest in deep learning over the past decade, multiple frameworks for writing neural networks have been developed:
  * [TensorFlow](https://www.tensorflow.org)
  * [PyTorch](https://www.pytorch.org)
  * [Keras](), which isn't a standalone library but rather a high-level user interface that simplifies the process of writing neural nets in TensorFlow and PyTorch.